In [98]:
import cv2
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
import pickle
from scipy.spatial.distance import cdist

# Path to the folder containing all the images
path = "/Users/michelle/Desktop/W281/final_project/rendered_256x256/256x256/sketch/tx_000100000000"

# List all the subdirectories (i.e., categories) in the main folder
categories = os.listdir(path)
categories = [x for x in categories if x != '.DS_Store']
len(categories)

125

In [99]:
%%time
sift = cv2.SIFT_create(nfeatures=200, nOctaveLayers=4, contrastThreshold=0.04, edgeThreshold=15, sigma=1.0)

# Define a list to store the descriptors of all the images
descriptors = []

# Define a list to store the corresponding labels of all the images
labels = []

# Loop through all the categories
for i, category in enumerate(categories):
    # Get the path to the category folder
    category_path = os.path.join(path, category)
    # List all the images in the category folder
    images = os.listdir(category_path)
    # Loop through all the images in the category folder
    for image in images:
        # Get the path to the image
        image_path = os.path.join(category_path, image)
        # Read the image
        img = cv2.imread(image_path)
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Get the keypoints and descriptors using SIFT
        kp, des = sift.detectAndCompute(gray, None)
        # Reduce the descriptor size to 64 dimensions for speed
        des = des[:, :64]
        # Store the descriptors in the descriptors list
        descriptors.append(des.flatten())
        # Store the label i
        labels.append(i)
        
        if len(descriptors) != len(labels):
            print(i)
            print(image_path)
            break

CPU times: user 20min 4s, sys: 15min 10s, total: 35min 14s
Wall time: 21min 55s


In [100]:
# standardize vector length
max_len = max([len(x) for x in descriptors])
new_arr = [np.pad(x, (0, (max_len-len(x)))) for x in descriptors]
#new_descriptors = [x[:min_length] for x in descriptors]

In [ ]:
%%time
X_train, X_test, Y_train, Y_test = train_test_split(new_arr, labels, test_size = 0.2, random_state = 1)
model_svm = SVC(C = 30, random_state = 0)
model_svm.fit(X_train, Y_train)

In [ ]:
%%time
# Evaluate the model on the testing set
accuracy = model_svm.score(X_test, Y_test)

print("Accuracy:", accuracy)

In [ ]:
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(model_svm, f)

In [ ]:
%%time
model_svm_linear = SVC(kernel = 'linear', C = 20, random_state = 0)
model_svm_linear.fit(X_train, Y_train)

In [ ]:
%%time
# Evaluate the model on the testing set
accuracy_linear = model_svm_linear.score(X_test, Y_test)

print("Accuracy:", accuracy)

In [ ]:
with open('svm_model_linear.pkl', 'wb') as f:
    pickle.dump(model_svm_linear, f)

### DONT RUN BELOW - KMEANS Approach

In [17]:
def kmean_bow(all_descriptors, num_cluster):
    """ run kmeans on the descriptors """
    bow_dict = []

    kmeans = KMeans(n_clusters = num_cluster)
    kmeans.fit(all_descriptors)

    bow_dict = kmeans.cluster_centers_

    return bow_dict


def create_feature_bow(image_descriptors, BoW, num_cluster):

    X_features = []

    for i in range(len(image_descriptors)):
        features = np.array([0] * num_cluster)

        if image_descriptors[i] is not None:
            distance = cdist(image_descriptors[i], BoW)
            argmin = np.argmin(distance, axis = 1)

            for j in argmin:
                features[j] += 1
        X_features.append(features)

    return X_features

all_descriptors = []
for descriptor in descriptors:
    if descriptor is not None:
        for des in descriptor:
            all_descriptors.append(des)
            
num_cluster = 20
BoW = kmean_bow(all_descriptors, num_cluster)
X_features = create_feature_bow(descriptors, BoW, num_cluster)